# Download SWOT Pixel Cloud products from hydroweb.next and extract information of interest for your study in a Zarr (zcollection) Database for future use


## Setting the spatial zone and the time period of interest
Using a geopackage layer, preliminary created with, e.g. QGIS, to limit data download and database

In [ ]:
from pixcdust.downloaders.hydroweb_next import PixCDownloader
import geopandas as gpd
from datetime import datetime

In [ ]:
# reading the area of interest polygon (could have been set)
gdf_geom = gpd.read_file('/home/hysope2/STUDIES/SWOT_Panama/DATA/aoi.gpkg')
geom = gpd.array.to_wkt(gdf_geom.geometry.values)[0]

# Limiting time period
dates = (
    datetime(2023,4,6),
    datetime(2023,4,8),
)


## Let's download matching data.
This will unfortunately lead to downloading many big files (that will be removed later). This is the only way right, but the hydroweb.next team is working on improving that.

In [ ]:
pixcdownloader = PixCDownloader(
    geom,
    dates,
    verbose=0,
    path_download='/tmp/pixc',
    )
pixcdownloader.download()

## Now we have all necessary files, let us extract key variables within area of interest in a Zarr (zcollection) database.
This Zarr partionned format is very efficient for time analysis, but is not currently accessible in GIS softwares such as QGIS
We are using the same geodataframe to limit the data to the area of interest

In [ ]:
from pixcdust.converters.zarr import PixCNc2ZarrConverter
from glob import glob

In [ ]:
# nb: it is important to provide files in a time strictly ascending or descending order, use the sorted function for this
pixc = PixCNc2ZarrConverter(
            sorted(glob(pixcdownloader.path_download+'/*/*nc')),
            "/tmp/my_awesome_pixc_zarr",
            variables=['height', 'sig0', 'classification'],
            area_of_interest=gdf_geom,
            mode='o',
        )
pixc.database_from_nc()

database has been succesfully created, we can remove the raw files

In [ ]:
import shutil
shutil.rmtree('/tmp/pixc')

# Let us use our Zarr Database
previous steps are not necessary

Now we can open this database in a xarray, or dataframe, or GeoDataFrame

In [ ]:
from pixcdust.readers.zarr import PixCZarrReader
import datetime

pixc_read = PixCZarrReader(
    "/tmp/my_awesome_pixc_zarr"
)
pixc_read.read((datetime.datetime(2023,4,10), datetime.datetime(2023,4,12)))
pixc_read.data

In [ ]:
gdf_pixc = pixc_read.to_geodataframe()
gdf_pixc

Enjoy!